### Clean Data from Eurostat

In [ ]:
import pandas as pd

# Create a DataFrame and get a first look
df = pd.read_csv("../data/raw/eurostat_demo_gind_DE_2010_2024.csv")
df.head()

,DATAFLOW,LAST UPDATE,freq,indic_de,geo,TIME_PERIOD,OBS_VALUE,OBS_FLAG,CONF_STATUS
0,ESTAT:DEMO_GIND(1.0),14/10/25 23:00:00,Annual,Net migration plus statistical adjustment,Germany,2010,130166.0,NaN,NaN
1,ESTAT:DEMO_GIND(1.0),14/10/25 23:00:00,Annual,Net migration plus statistical adjustment,Germany,2011,295478.0,b,NaN
2,ESTAT:DEMO_GIND(1.0),14/10/25 23:00:00,Annual,Net migration plus statistical adjustment,Germany,2012,391884.0,NaN,NaN
3,ESTAT:DEMO_GIND(1.0),14/10/25 23:00:00,Annual,Net migration plus statistical adjustment,Germany,2013,455473.0,NaN,NaN
4,ESTAT:DEMO_GIND(1.0),14/10/25 23:00:00,Annual,Net migration plus statistical adjustment,Germany,2014,583503.0,NaN,NaN


#### Plan for Cleaning
1. Drop metadata columns (DATAFLOW, LAST UPDATE, freq, geo, OBS_FLAG, CONF_STATUS)
2. Format "OBS_VALUE" numeric
3. Change the structure

In [17]:
# Drop metadata
drop_cols = ["DATAFLOW", "LAST UPDATE", "freq", "geo", "CONF_STATUS", "OBS_FLAG"]
df_dropped = df.drop(columns = [c for c in drop_cols if c in df.columns])

# Numeric conversion 
df_dropped["OBS_VALUE"] = pd.to_numeric(df_dropped["OBS_VALUE"], errors="coerce")
df_dropped["TIME_PERIOD"] = pd.to_numeric(df_dropped["TIME_PERIOD"], errors="coerce")

df_dropped.head()


,indic_de,TIME_PERIOD,OBS_VALUE
0,Net migration plus statistical adjustment,2010,130166.0
1,Net migration plus statistical adjustment,2011,295478.0
2,Net migration plus statistical adjustment,2012,391884.0
3,Net migration plus statistical adjustment,2013,455473.0
4,Net migration plus statistical adjustment,2014,583503.0


#### Pivot
- Create a pivot DataFrame to later save as csv

In [18]:
# Create a pivot 
cleaned = (
    df_dropped.pivot(index="indic_de",
                     columns="TIME_PERIOD",
                     values="OBS_VALUE").sort_index(axis=1)
)

# Cleaner formatting
cleaned.columns.name = None
rename_map = {
    "Crude rate of net migration plus statistical adjustment": "net migration rate",
    "Natural change of population": "natural population change",
    "Net migration plus statistical adjustment": "net migration",
    "Population on 1 January - total": "total population (1 Jan)"
}
cleaned = cleaned.rename(index = rename_map)


cleaned.head()

,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
indic_de,,,,,,,,,,,,,,,
net migration rate,1.6,3.7,4.9,5.6,7.2,14.3,5.6,5.1,4.8,3.7,2.4,3.7,2.5,8.1,5.4
natural population change,-180821.0,-189643.0,-196038.0,-211756.0,-153429.0,-187625.0,-118761.0,-147371.0,-167351.0,-161430.0,-212428.0,-228195.0,-327522.0,-335217.0,-330641.0
net migration,130166.0,295478.0,391884.0,455473.0,583503.0,1165772.0,464730.0,418069.0,394213.0,308928.0,200748.0,310288.0,208899.0,672761.0,451736.0
total population (1 Jan),81802257.0,80222065.0,80327900.0,80523746.0,80767463.0,81197537.0,82175684.0,82521653.0,82792351.0,83019213.0,83166711.0,83155031.0,83237124.0,83118501.0,83456045.0


In [19]:
# Save as csv 
cleaned.to_csv("../data/processed/eurostat_cleaned_2010_2024.csv")